#Enabling GPU

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# Importing the Libraries

In [ ]:
!pip install bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 15.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=0accc47c1b5f8013f149114c2ef74a59f46764567936373d23e300ff9d52609b
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [ ]:
# Importing the libraries for preprocessing
import nltk
import string
import re
import math
from tensorflow import keras
from bnlp import NLTKTokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
If you are using version <=3.3.2 please specify bnlp_toolkit with exact version, otherwise it will raises error in the upcoming version. 
To migrate feel free to checkout the newer version (4.0.0). It will release soon as beta.


# Preprocessing

In [ ]:
def pre_processing(documents):
  '''This function calls all the pre-processing submodules'''

  documents = remove_english(documents)
  documents = remove_whitespace(documents)
  documents = remove_hashtag(documents)
  documents = remove_urls(documents)
  documents = remove_special_character(documents)
  documents = remove_newline(documents)
  documents = bnlp_word(documents)
  # documents = bnlp_sentence(documents)
  # documents = from_text_into_sequence(documents)
  return documents

In [ ]:
#-----Text to Sequence-----#
def from_text_into_sequence(document):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(document)
  return tokenizer.texts_to_sequences(document)

In [ ]:
#-----Word Tokenization-----#
def bnlp_word(data):
  '''This function removes HashTag # from the entire dataset'''
  bnlp_data = []
  bnltk = NLTKTokenizer()

  for line in data:
    bnlp_data.append(bnltk.word_tokenize(line))

  return bnlp_data

In [ ]:
#-----Sentence Tokenization-----#
def bnlp_sentence(data):
  '''This function removes HashTag # from the entire dataset'''
  bnlp_data = []
  bnltk = NLTKTokenizer()

  for line in data:
    bnlp_data.append(bnltk.sentence_tokenize(line))

  return bnlp_data

In [ ]:
#-----Removing English Alphabets-----#
def remove_english(data):
  '''This function removes english from text '''
  english_removed_data = []
  for line in data:
    english_removed_data.append(re.sub('[A-Za-z]', '', line))
  return english_removed_data

In [ ]:
#-----Removing Whitespace-----#
def remove_whitespace(data):
  '''This function removes multiple whitespaces and replace them with a single whitespace'''
  whitespace_removed_data = []

  for line in data:
    whitespace_removed_data.append(" ".join(line.split()))

  return whitespace_removed_data

In [ ]:
#-----Removing Hashtag-----#
def remove_hashtag(data):
  '''This function removes HashTag # from the entire dataset'''
  hashtag_removed_data = []

  # map hashtag to space
  translator = str.maketrans('#', ' '*len('#'), '')

  for line in data:
    hashtag_removed_data.append(line.translate(translator))

  return hashtag_removed_data

In [ ]:
#-----Removing URLs-----#
def remove_urls(data):
  url_removed_data = []

  for line in data:
    url_removed_data.append(re.sub('http[s]?://\S+', '', line))

  return url_removed_data

In [ ]:
#-----Removing Special Character-----#
def remove_special_character(data):
  special_character_removed_data = []

  for line in data:
    special_character_removed_data.append(re.sub('[!,*)@%(&$_?.^/<|>+-{=~`}]', '', line))

  return special_character_removed_data

In [ ]:
#-----Removing Newline-----#
def remove_newline(data):
  newline_removed_data = []

  for line in data:
    newline_removed_data.append(re.sub('\n', '', line))

  return newline_removed_data

In [ ]:
def combine_tokens(data):
  return [s for S in data for s in S]

In [ ]:
def identity(X):
  return X

In [ ]:
def join_words(doc):
  tmp_doc = []
  for sent in doc:
    tmp_doc.append(' '.join(sent))

  return tmp_doc

# Mounting data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project_Bangla_Author_Profiling/Project/BN-AuthProf_Dataset

/content/drive/.shortcut-targets-by-id/1jftPvn0FbUItDzJ1LSOelGKwEFTbAqpJ/Project_Bangla_Author_Profiling/Project/BN-AuthProf_Dataset


In [ ]:
# copying the dataset into colab local directory (for faster execution)
zip_path = 'BN-AuthProf_Dataset.zip'
!cp "{zip_path}" .
!unzip -q BN-AuthProf_Dataset.zip

cp: 'BN-AuthProf_Dataset.zip' and './BN-AuthProf_Dataset.zip' are the same file
replace BN-AuthProf_Dataset/Exp_Dataset/Age/Train/493.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


#Gender Classification

##Extracting Label

In [ ]:
# Getting the #Truth file in to an array

train_truth_data = []
with open('BN-AuthProf_Dataset/Exp_Dataset/Gender/Train/#Truth.txt', 'r') as truth_file:
  for line in truth_file:
    train_truth_data.append(line)

dev_truth_data = []
with open('BN-AuthProf_Dataset/Exp_Dataset/Gender/Dev/#Truth.txt', 'r') as truth_file:
  for line in truth_file:
    dev_truth_data.append(line)

In [ ]:
# Train Truth Data
train_ids = []
train_gender = []

for line in train_truth_data:
  splitted_line = line.strip().split(':::', 3)
  train_ids.append(splitted_line[0])
  train_gender.append(splitted_line[1])


train_gender_dictionary = { 'M' :0, 'F' :1}
train_gender_labels= [train_gender_dictionary[i] for i in train_gender]

# Dev Truth Data
dev_ids = []
dev_gender = []

for line in dev_truth_data:
  splitted_line = line.strip().split(':::', 3)
  dev_ids.append(splitted_line[0])
  dev_gender.append(splitted_line[1])


dev_gender_dictionary = { 'M' :0, 'F' :1}
dev_gender_labels= [dev_gender_dictionary[i] for i in dev_gender]




##Extracting Data

In [ ]:
# Train Data
data_train = []
train_data_trial = []

for i in range(0,454):
  with open('BN-AuthProf_Dataset/Exp_Dataset/Gender/Train/{0}.txt'.format(i+1),'r', encoding='utf-8') as f:
    for line in f:
      data_train.append(line)

  len_of_data_train = len(data_train)
  train_data_trial.append(data_train[:len_of_data_train])
  data_train.clear()

In [ ]:
# Dev Data
data_dev = []
dev_data_trial = []

for i in range(0,300):
  with open('BN-AuthProf_Dataset/Exp_Dataset/Gender/Dev/{0}.txt'.format(i+1),'r', encoding='utf-8') as f:
    for line in f:
      data_dev.append(line)

  len_of_data_dev = len(data_dev)
  dev_data_trial.append(data_dev[:len_of_data_dev])
  data_dev.clear()

In [ ]:
# Test Data
data_test = []
test_data_trial = []

for i in range(0,300):
  with open('BN-AuthProf_Dataset/Exp_Dataset/Gender/Test/{0}.txt'.format(i+1),'r', encoding='utf-8') as f:
    for line in f:
      data_test.append(line)

  len_of_data_test = len(data_test)
  test_data_trial.append(data_test[:len_of_data_test])
  data_test.clear()

## Preprocessing train data, test data and dev data

In [ ]:
test_data = []
dev_data = []
train_data = []

for line in test_data_trial:
  test_data.append(combine_tokens(pre_processing(line)))

for line in dev_data_trial:
  dev_data.append(combine_tokens(pre_processing(line)))

for line in train_data_trial:
  train_data.append(combine_tokens(pre_processing(line)))



## NB

In [ ]:
# importing the Libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB

In [ ]:
def vec_tfidf(tfidf = True):

  if tfidf:
    vec = TfidfVectorizer(preprocessor = identity, analyzer='word',
                          tokenizer = identity, ngram_range = (2,4))

  else:
    vec = CountVectorizer(preprocessor = identity, analyzer='char',
                          tokenizer = identity, ngram_range = (2,4))
  return vec

In [ ]:
#######   Char Analyzer     ######

vec = vec_tfidf(tfidf = False)
 # combines the vectorizer with the Naive Bayes classifier
classifier = Pipeline([('vec', vec),
                         ('cls',MultinomialNB())])
%time

classifier.fit(join_words(train_data), train_gender_labels)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.78 µs


Pipeline(steps=[('vec',
                 CountVectorizer(analyzer='char', ngram_range=(2, 4),
                                 preprocessor=<function identity at 0x7f2dd452f1c0>,
                                 tokenizer=<function identity at 0x7f2dd452f1c0>)),
                ('cls', MultinomialNB())])

In [ ]:
prediction = classifier.predict(join_words(test_data))

print("Naive Bayes Accuracy = ", accuracy_score(test_gender_labels, prediction))
print()

print(classification_report(test_gender_labels, prediction, labels=classifier.classes_, digits=3))

Naive Bayes Accuracy =  0.7966666666666666

              precision    recall  f1-score   support

           0      0.923     0.797     0.856       227
           1      0.558     0.795     0.655        73

    accuracy                          0.797       300
   macro avg      0.741     0.796     0.756       300
weighted avg      0.834     0.797     0.807       300



# Age Classification

##Extracting truth data


In [ ]:
train_truth_data_age = []
with open('BN-AuthProf_Dataset/Exp_Dataset/Age/Train/#Truth.txt', 'r') as truth_file:
  for line in truth_file:
    train_truth_data_age.append(line)

dev_truth_data_age = []
with open('BN-AuthProf_Dataset/Exp_Dataset/Age/Dev/#Truth.txt', 'r') as truth_file:
  for line in truth_file:
    dev_truth_data_age.append(line)

In [ ]:
# Train Truth Data
train_ids = []
train_age = []
train_gender = []


for line in train_truth_data_age:
  splitted_line = line.strip().split(':::', 3)
  train_ids.append(splitted_line[0])
  train_gender.append(splitted_line[1])
  train_age.append(splitted_line[2])


train_age_dictionary = { '18-24' :0, '25-34' :1, '35-49' :2, '50-xx' :3}
train_age_labels= [train_age_dictionary[i] for i in train_age]

# Dev Truth Data
dev_ids = []
dev_age = []
dev_gender = []

for line in dev_truth_data_age:
  splitted_line = line.strip().split(':::', 3)
  dev_ids.append(splitted_line[0])
  dev_gender.append(splitted_line[1])
  dev_age.append(splitted_line[2])


dev_age_dictionary = { '18-24' :0, '25-34' :1, '35-49' :2, '50-xx' :3}
dev_age_labels= [dev_age_dictionary[i] for i in dev_age]

dev_gender_dictionary = { 'M' :0, 'F' :1}
dev_gender_labels= [dev_gender_dictionary[i] for i in dev_gender]

##Extracting Data

In [ ]:
# Train Data
data_train = []
train_data_trial = []

for i in range(0,508):
  with open('BN-AuthProf_Dataset/Exp_Dataset/Age/Train/{0}.txt'.format(i+1),'r', encoding='utf-8') as f:
    for line in f:
      data_train.append(line)

  len_of_data_train = len(data_train)
  train_data_trial.append(data_train[:len_of_data_train])
  data_train.clear()


In [ ]:
len(train_data_trial)

508

In [ ]:
# Dev Data
data_dev = []
dev_data_trial = []

for i in range(0,300):
  with open('BN-AuthProf_Dataset/Exp_Dataset/Age/Dev/{0}.txt'.format(i+1),'r', encoding='utf-8') as f:
    for line in f:
      data_dev.append(line)

  len_of_data_dev = len(data_dev)
  dev_data_trial.append(data_dev[:len_of_data_dev])
  data_dev.clear()

In [ ]:
len(dev_data_trial)

300

In [ ]:
#Test Data
data_test = []
test_data_trial = []

for i in range(0,300):
  with open('BN-AuthProf_Dataset/Exp_Dataset/Age/Test/{0}.txt'.format(i+1),'r', encoding='utf-8') as f:
    for line in f:
      data_test.append(line)

  len_of_data_test = len(data_test)
  test_data_trial.append(data_test[:len_of_data_test])
  data_test.clear()

In [ ]:
len(test_data_trial)

300

## Preprocessing train data, test data and dev data

In [ ]:
test_data = []
dev_data = []
train_data = []
for line in test_data_trial:
  test_data.append(combine_tokens(pre_processing(line)))

for line in dev_data_trial:
  dev_data.append(combine_tokens(pre_processing(line)))

for line in train_data_trial:
  train_data.append(combine_tokens(pre_processing(line)))



## NB

In [ ]:
#importing the Libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB

In [ ]:
def vec_tfidf(tfidf = True):

  if tfidf:
    vec = TfidfVectorizer(preprocessor = identity, analyzer='word',
                          tokenizer = identity, ngram_range = (2,4))

  else:
    vec = CountVectorizer(preprocessor = identity, analyzer='word',
                          tokenizer = identity, ngram_range = (2,3))
  return vec

In [ ]:
#######   Char Analyzer     ######

vec = vec_tfidf(tfidf = False)
 # combines the vectorizer with the Naive Bayes classifier
classifier = Pipeline([('vec', vec),
                         ('cls',MultinomialNB())])
%time

classifier.fit(train_data, train_age_labels)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


Pipeline(steps=[('vec',
                 CountVectorizer(ngram_range=(2, 3),
                                 preprocessor=<function identity at 0x7f2e632c1360>,
                                 tokenizer=<function identity at 0x7f2e632c1360>)),
                ('cls', MultinomialNB())])

In [ ]:
# prediction = classifier.predict(join_words(dev_data))
prediction = classifier.predict(test_data)

print("Naive Bayes Accuracy = ", accuracy_score(test_age_labels, prediction))
print()

print(classification_report(test_age_labels, prediction, labels=classifier.classes_, digits=3))

Naive Bayes Accuracy =  0.91

              precision    recall  f1-score   support

           0      0.729     1.000     0.843        43
           1      0.982     0.850     0.911       127
           2      0.939     0.939     0.939        82
           3      0.918     0.938     0.928        48

    accuracy                          0.910       300
   macro avg      0.892     0.932     0.905       300
weighted avg      0.924     0.910     0.912       300

